# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [40]:
import pandas as pd
import numpy as np
import math
import json
import matplotlib.pyplot as plt
import seaborn as sns

# read in the json files
portfolio_raw = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile_raw = pd.read_json('data/profile.json', orient='records', lines=True)
transcript_raw = pd.read_json('data/transcript.json', orient='records', lines=True)

## Data understanding



**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)


** -> Data is cleaned and ready to be used**

In [41]:
portfolio_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reward      10 non-null     int64 
 1   channels    10 non-null     object
 2   difficulty  10 non-null     int64 
 3   duration    10 non-null     int64 
 4   offer_type  10 non-null     object
 5   id          10 non-null     object
dtypes: int64(3), object(3)
memory usage: 612.0+ bytes


In [59]:
portfolio_raw

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


In [42]:
# portfolio_raw['channels'].explode().unique()

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**-> the NoneType gender will be removed from profile dataset and storage in a new dataset**

In [43]:
profile_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            14825 non-null  object 
 1   age               17000 non-null  int64  
 2   id                17000 non-null  object 
 3   became_member_on  17000 non-null  int64  
 4   income            14825 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 664.2+ KB


**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

In [44]:
transcript_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   person  306534 non-null  object
 1   event   306534 non-null  object
 2   value   306534 non-null  object
 3   time    306534 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 9.4+ MB


In [45]:
transcript_raw['event'].value_counts()

event
transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: count, dtype: int64

In [46]:
{[*x][0] for x in transcript_raw['value']}

{'amount', 'offer id', 'offer_id'}

### Data Cleaning and Formatting Plan

#### **Objective**
- Clean and standardize the dataset while handling null values appropriately.

---

#### **Profile Dataset**
1. Split the `profile` dataset into two subsets:
   - One containing rows where the `gender` is `none`.
   - Another containing rows with all other `gender` values.

2. Convert the `became_member_on` column to a standardized **datetime** format for consistency and easier analysis.

---

#### **Portfolio Dataset**
- Update column names to make them more descriptive and easier to read.

---

#### **Transcript Dataset**
- Modify the keys within the `value` dictionaries by replacing spaces (`' '`) with underscores (`'_'`) to ensure uniformity and avoid potential issues in further processing.

---

In [47]:
profile = profile_raw.copy(deep=True)

# Convert the 'became_member_on' column to a datetime format
profile['became_member_on'] = pd.to_datetime(profile['became_member_on'], format='%Y%m%d')

# Create a new column with only the year and month of the membership
profile['bec_memb_year_month'] = pd.to_datetime(profile['became_member_on'], format='%Y%m%d').dt.strftime('%Y-%m')

In [48]:
profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   gender               14825 non-null  object        
 1   age                  17000 non-null  int64         
 2   id                   17000 non-null  object        
 3   became_member_on     17000 non-null  datetime64[ns]
 4   income               14825 non-null  float64       
 5   bec_memb_year_month  17000 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 797.0+ KB


In [49]:
portfolio = portfolio_raw.copy()
portfolio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reward      10 non-null     int64 
 1   channels    10 non-null     object
 2   difficulty  10 non-null     int64 
 3   duration    10 non-null     int64 
 4   offer_type  10 non-null     object
 5   id          10 non-null     object
dtypes: int64(3), object(3)
memory usage: 612.0+ bytes


In [50]:
# Standardisation of column names
transcript = transcript_raw.copy(deep=True)
transcript['event'] = transcript['event'].str.replace(' ','_')

# Fixing the dicts key names
def fix_offer_id(value):
    if isinstance(value, dict) and 'offer id' in value:
        value['offer_id'] = value.pop('offer id')
    return value

transcript['value'] = transcript['value'].apply(fix_offer_id)

# Normalizing the value column and create a new dataframe
value_df = pd.json_normalize(transcript['value']) 
transcript_b = pd.concat([transcript, value_df], axis=1).drop('value', axis=1)

port_id = {
    'ae264e3637204a6fb9bb56bc8210ddfd': 'ofr_A',
    '4d5c57ea9a6940dd891ad53e9dbe8da0': 'ofr_B',
    '3f207df678b143eea3cee63160fa8bed': 'ofr_C',
    '9b98b8c7a33c4b65b9aebfe6a799e6d9': 'ofr_D',
    '0b1e1539f2cc45b7b9fa7c272da2e1d7': 'ofr_E',
    '2298d6c36e964ae4a3e7e9706d1fb8c2': 'ofr_F',
    'fafdcd668e3743c1bb461111dcafc2a4': 'ofr_G',
    '5a8bc65990b245e5a138643cd4eb9837': 'ofr_H',
    'f19421c1d4aa40978ebb69ca19b0e20d': 'ofr_I',
    '2906b810c7d4411798c6938adc9daaa5': 'ofr_J'
}

transcript_b['ofr_id_short'] = transcript_b['offer_id'].map(port_id)
transcript_b['ofr_id_short'] = transcript_b['ofr_id_short'].fillna('tran')
transcript_b['amount'] = transcript_b['amount'].fillna(0)
transcript_b['reward'] = transcript_b['reward'].fillna(0)

transcript_b = transcript_b.drop(columns = ['offer_id'])

transcript_b['tag'] = (
    transcript_b.groupby(['person', 'ofr_id_short', 'event'])
    .cumcount()
)

transactions = transcript_b[transcript_b['ofr_id_short'] == 'tran'].copy()
events = transcript_b[transcript_b['ofr_id_short'] != 'tran'].copy()

events = events.groupby(['person', 'ofr_id_short', 'event', 'tag']).agg(
    time=('time', 'mean'),
    reward=('reward', 'sum'),
).unstack([2]).reset_index()

events.columns = ['_'.join(col).strip('_') for col in events.columns.to_flat_index()]

events = events.drop(columns=['reward_offer_received',	'reward_offer_viewed'])

events['time_offer_completed'] = events['time_offer_completed'].fillna(np.inf)
events['time_offer_received'] = events['time_offer_received'].fillna(np.inf)
events['time_offer_viewed'] = events['time_offer_viewed'].fillna(np.inf)

events['to_vr'] = events['time_offer_viewed'] - events['time_offer_received']
events['to_cv'] = events['time_offer_completed'] - events['time_offer_viewed'] 
events['to_cr'] = events['time_offer_completed'] - events['time_offer_received']

events['curiosity_vr'] = (1 / (events['to_vr'] + 1)) * 100
events['eagerness_cv'] = (1 / ((events['to_cv']) + 1)) * 100
events['overall_cr'] = (1 / (events['to_cr'] + 1)) * 100

events['influence'] = ((events['time_offer_completed'] != np.inf) & (events['time_offer_completed'] >= events['time_offer_viewed'])).astype(int) 
events['ext_influence'] = ((events['time_offer_viewed'] != np.inf) & (events['time_offer_completed'] == events['time_offer_viewed'])).astype(int) 


events = events.drop(columns=['time_offer_completed', 'time_offer_received', 'time_offer_viewed', 'to_vr', 'to_cv', 'to_cr'	]).round(1)



transac_person_sumary = transactions.groupby(['person']).agg(
    # tran_count=('amount', 'count'),
    tran_amoun_tot =('amount', 'sum'),
    tran_amoun_mean=('amount', 'mean'),
    tran_amoun_max=('amount', 'max'),
    tran_amoun_min=('amount', 'min'),
    # tran_amount_std = ('amount', 'std')
).reset_index().round(1)

event_count = transcript_b.groupby(['person', 'event']).agg(
    cnt = ('event', 'count'),
    avg_time = ('time', 'mean'),
    max_time = ('time', 'max'),
    min_time = ('time', 'min')
    
    ).unstack(1).reset_index()

event_count.columns = ['_'.join(col).strip('_') for col in event_count.columns.to_flat_index()]

# events_merged = events.merge(transactions, left_on=['person', 'time_offer_completed'], right_on=['person', 'time'], how='left')

person_activity = events.merge(transac_person_sumary, on='person', how='left')

person_activity = person_activity.merge(event_count, on='person', how='left').fillna(0)

person_activity_profile = person_activity.merge(profile, left_on='person', right_on='id', how='left').drop(columns=['id'])



person_activity_profile.to_csv('person_activity_profile.csv')

person_activity_profile



,person,ofr_id_short,tag,reward_offer_completed,curiosity_vr,eagerness_cv,overall_cr,influence,ext_influence,tran_amoun_tot,...,max_time_transaction,min_time_offer_completed,min_time_offer_received,min_time_offer_viewed,min_time_transaction,gender,age,became_member_on,income,bec_memb_year_month
0,0009655768c64bdeb2e877511632db8f,ofr_C,0,0.0,2.7,0.0,0.0,0,0,127.6,...,696.0,414.0,168.0,192.0,228.0,M,33,2017-04-21,72000.0,2017-04
1,0009655768c64bdeb2e877511632db8f,ofr_G,0,2.0,2.7,-9.1,4.0,0,0,127.6,...,696.0,414.0,168.0,192.0,228.0,M,33,2017-04-21,72000.0,2017-04
2,0009655768c64bdeb2e877511632db8f,ofr_H,0,0.0,4.0,0.0,0.0,0,0,127.6,...,696.0,414.0,168.0,192.0,228.0,M,33,2017-04-21,72000.0,2017-04
3,0009655768c64bdeb2e877511632db8f,ofr_I,0,5.0,2.0,-2.4,14.3,0,0,127.6,...,696.0,414.0,168.0,192.0,228.0,M,33,2017-04-21,72000.0,2017-04
4,0009655768c64bdeb2e877511632db8f,ofr_J,0,2.0,0.0,-0.0,100.0,0,0,127.6,...,696.0,414.0,168.0,192.0,228.0,M,33,2017-04-21,72000.0,2017-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76272,ffff82501cea40309d5fdd7edcca4a07,ofr_E,0,5.0,14.3,4.0,3.2,1,0,226.1,...,648.0,60.0,0.0,6.0,60.0,F,45,2016-11-25,62000.0,2016-11
76273,ffff82501cea40309d5fdd7edcca4a07,ofr_G,0,2.0,14.3,1.8,1.6,1,0,226.1,...,648.0,60.0,0.0,6.0,60.0,F,45,2016-11-25,62000.0,2016-11
76274,ffff82501cea40309d5fdd7edcca4a07,ofr_J,0,2.0,5.3,3.2,2.0,1,0,226.1,...,648.0,60.0,0.0,6.0,60.0,F,45,2016-11-25,62000.0,2016-11
76275,ffff82501cea40309d5fdd7edcca4a07,ofr_J,1,2.0,14.3,100.0,14.3,1,1,226.1,...,648.0,60.0,0.0,6.0,60.0,F,45,2016-11-25,62000.0,2016-11


In [51]:
person_activity_profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76277 entries, 0 to 76276
Data columns (total 34 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   person                    76277 non-null  object        
 1   ofr_id_short              76277 non-null  object        
 2   tag                       76277 non-null  int64         
 3   reward_offer_completed    76277 non-null  float64       
 4   curiosity_vr              76277 non-null  float64       
 5   eagerness_cv              76277 non-null  float64       
 6   overall_cr                76277 non-null  float64       
 7   influence                 76277 non-null  int64         
 8   ext_influence             76277 non-null  int64         
 9   tran_amoun_tot            76277 non-null  float64       
 10  tran_amoun_mean           76277 non-null  float64       
 11  tran_amoun_max            76277 non-null  float64       
 12  tran_amoun_min    

In [52]:
transcript_b[transcript_b['person'] == 'ffed75d3abc64b488982f50ed12878b5']

,person,event,time,amount,reward,ofr_id_short,tag
2970,ffed75d3abc64b488982f50ed12878b5,offer_received,0,0.0,0.0,ofr_B,0
20733,ffed75d3abc64b488982f50ed12878b5,offer_viewed,18,0.0,0.0,ofr_B,0
113829,ffed75d3abc64b488982f50ed12878b5,offer_received,336,0.0,0.0,ofr_A,0
130780,ffed75d3abc64b488982f50ed12878b5,offer_viewed,348,0.0,0.0,ofr_A,0


In [53]:
transactions['person'].unique()

array(['02c083884c7d45b39cc68e1314fec56c',
       '9fa9ae8f57894cc9a3b8a9bbe0fc1b2f',
       '54890f68699049c2a04d415abc25e717', ...,
       '7718656997f3453db0f5aeca9cd35240',
       '54463e5d95124b7fb3133fc1eae71952',
       '0c027f5f34dd4b9eba0a25785c611273'], shape=(16578,), dtype=object)

In [54]:
# Check if all values in transactions['person'] are in events['person']
all_in_events = transactions['person'].isin(events['person']).all()

# Check if all values in events['person'] are in transactions['person']
all_in_transactions = events['person'].isin(transactions['person']).all()

print(f"All values in transactions['person'] are in events['person']: {all_in_events}")
print(f"All values in events['person'] are in transactions['person']: {all_in_transactions}")

All values in transactions['person'] are in events['person']: False
All values in events['person'] are in transactions['person']: False


In [55]:
person_activity[person_activity['person'] == '0020c2b971eb4e9188eac86d93036a77']  

,person,ofr_id_short,tag,reward_offer_completed,curiosity_vr,eagerness_cv,overall_cr,influence,ext_influence,tran_amoun_tot,...,avg_time_offer_viewed,avg_time_transaction,max_time_offer_completed,max_time_offer_received,max_time_offer_viewed,max_time_transaction,min_time_offer_completed,min_time_offer_received,min_time_offer_viewed,min_time_transaction
12,0020c2b971eb4e9188eac86d93036a77,ofr_A,0,0.0,0.0,0.0,0.0,0,0,196.9,...,366.0,348.75,510.0,504.0,660.0,708.0,54.0,0.0,12.0,54.0
13,0020c2b971eb4e9188eac86d93036a77,ofr_B,0,10.0,5.3,1.2,1.0,1,0,196.9,...,366.0,348.75,510.0,504.0,660.0,708.0,54.0,0.0,12.0,54.0
14,0020c2b971eb4e9188eac86d93036a77,ofr_G,0,2.0,7.7,2.3,1.8,1,0,196.9,...,366.0,348.75,510.0,504.0,660.0,708.0,54.0,0.0,12.0,54.0
15,0020c2b971eb4e9188eac86d93036a77,ofr_G,1,2.0,0.0,-0.0,0.6,0,0,196.9,...,366.0,348.75,510.0,504.0,660.0,708.0,54.0,0.0,12.0,54.0
16,0020c2b971eb4e9188eac86d93036a77,ofr_H,0,0.0,0.6,0.0,0.0,0,0,196.9,...,366.0,348.75,510.0,504.0,660.0,708.0,54.0,0.0,12.0,54.0


In [56]:
person_activity[person_activity['cnt_transaction'] == 0]  

,person,ofr_id_short,tag,reward_offer_completed,curiosity_vr,eagerness_cv,overall_cr,influence,ext_influence,tran_amoun_tot,...,avg_time_offer_viewed,avg_time_transaction,max_time_offer_completed,max_time_offer_received,max_time_offer_viewed,max_time_transaction,min_time_offer_completed,min_time_offer_received,min_time_offer_viewed,min_time_transaction
486,01d7da27b8934ba1b3602a0153e4415f,ofr_D,0,0.0,1.1,0.0,0.0,0,0,0.0,...,396.0,0.0,0.0,576.0,666.0,0.0,0.0,0.0,168.0,0.0
487,01d7da27b8934ba1b3602a0153e4415f,ofr_E,0,0.0,0.0,0.0,0.0,0,0,0.0,...,396.0,0.0,0.0,576.0,666.0,0.0,0.0,0.0,168.0,0.0
488,01d7da27b8934ba1b3602a0153e4415f,ofr_G,0,0.0,100.0,0.0,0.0,0,0,0.0,...,396.0,0.0,0.0,576.0,666.0,0.0,0.0,0.0,168.0,0.0
489,01d7da27b8934ba1b3602a0153e4415f,ofr_H,0,0.0,100.0,0.0,0.0,0,0,0.0,...,396.0,0.0,0.0,576.0,666.0,0.0,0.0,0.0,168.0,0.0
490,01d7da27b8934ba1b3602a0153e4415f,ofr_H,1,0.0,14.3,0.0,0.0,0,0,0.0,...,396.0,0.0,0.0,576.0,666.0,0.0,0.0,0.0,168.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76209,ffd810f301ad42d6bfe98c4ba6ee4125,ofr_C,0,0.0,14.3,0.0,0.0,0,0,0.0,...,282.0,0.0,0.0,408.0,492.0,0.0,0.0,0.0,12.0,0.0
76210,ffd810f301ad42d6bfe98c4ba6ee4125,ofr_I,0,0.0,7.7,0.0,0.0,0,0,0.0,...,282.0,0.0,0.0,408.0,492.0,0.0,0.0,0.0,12.0,0.0
76211,ffd810f301ad42d6bfe98c4ba6ee4125,ofr_J,0,0.0,1.2,0.0,0.0,0,0,0.0,...,282.0,0.0,0.0,408.0,492.0,0.0,0.0,0.0,12.0,0.0
76238,ffed75d3abc64b488982f50ed12878b5,ofr_A,0,0.0,7.7,0.0,0.0,0,0,0.0,...,183.0,0.0,0.0,336.0,348.0,0.0,0.0,0.0,18.0,0.0


In [57]:
person_activity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76277 entries, 0 to 76276
Data columns (total 29 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   person                    76277 non-null  object 
 1   ofr_id_short              76277 non-null  object 
 2   tag                       76277 non-null  int64  
 3   reward_offer_completed    76277 non-null  float64
 4   curiosity_vr              76277 non-null  float64
 5   eagerness_cv              76277 non-null  float64
 6   overall_cr                76277 non-null  float64
 7   influence                 76277 non-null  int64  
 8   ext_influence             76277 non-null  int64  
 9   tran_amoun_tot            76277 non-null  float64
 10  tran_amoun_mean           76277 non-null  float64
 11  tran_amoun_max            76277 non-null  float64
 12  tran_amoun_min            76277 non-null  float64
 13  cnt_offer_completed       76277 non-null  float64
 14  cnt_of

In [58]:
sns.scatterplot(data=person_activity, x='time_offer_viewed', y='time_offer_completed', hue='ofr_id_short_x')

ValueError: Could not interpret value `time_offer_viewed` for `x`. An entry with this name does not appear in `data`.

In [ ]:
sns.scatterplot(data=person_activity[(person_activity['person'] == '0020c2b971eb4e9188eac86d93036a77')], x='time_offer_viewed', y='time_offer_completed', hue='ofr_id_short_x') #(person_activity['ofr_id_short_x'] == 'ofr_B') & 